# Options Visualizer

The goal of this notebook is to get a better grasp on the dynamics of nth-order Greeks.

## Plot One Example

First let's try building out what one plot would look like until we're satisfied with the results. Then, once it's easily scalable, we can build out the remaining plots.

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as si
import ipywidgets as widgets

def delta(S, K, T, r, sigma, option_type='call'):
    """calculate delta for european options"""
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    if option_type == 'call':
        return si.norm.cdf(d1)
    else:
        return si.norm.cdf(d1) - 1

def plot_delta(K=100, T=1, r=0.05, sigma=0.2, option_type='call', S_current=100):
    """plot delta as a function of stock price and show the current delta"""
    S_range = np.linspace(50, 150, 400)  # Range of stock prices
    delta_values = [delta(S, K, T, r, sigma, option_type) for S in S_range]
    current_delta = delta(S_current, K, T, r, sigma, option_type)

    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(S_range, delta_values, label='Delta', color='blue')
    ax.scatter(S_current, current_delta, color='red', zorder=5, label=f'Current Delta: {current_delta:.2f}')
    ax.axvline(S_current, color='red', linestyle='--', linewidth=0.5)
    ax.set_title('Delta vs Stock Price')
    ax.set_xlabel('Stock Price (S)')
    ax.set_ylabel('Delta')
    ax.legend()
    plt.grid(True)
    plt.show()

style = {'description_width': 'initial'}
widgets.interact(
    plot_delta,
    S_current=widgets.FloatSlider(value=100, min=50, max=150, step=1, description='Stock Price (S)', style=style),
    K=widgets.FloatSlider(value=100, min=50, max=150, step=1, description='Strike Price (K)', style=style),
    T=widgets.FloatSlider(value=1, min=0.01, max=2, step=0.01, description='Time to Expiry (T)', style=style),
    r=widgets.FloatSlider(value=0.05, min=0, max=0.2, step=0.01, description='Risk-Free Rate (r)', style=style),
    sigma=widgets.FloatSlider(value=0.2, min=0.01, max=1, step=0.01, description='Volatility (σ)', style=style),
    option_type=widgets.Dropdown(options=['call', 'put'], value='call', description='Option Type', style=style)
)


interactive(children=(FloatSlider(value=100.0, description='Strike Price (K)', max=150.0, min=50.0, step=1.0, …

<function __main__.plot_delta(K=100, T=1, r=0.05, sigma=0.2, option_type='call', S_current=100)>

## First and Second Order Greeks

I think this now works fantastic, let's move on to building out the rest of the visualizations!

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as si
import ipywidgets as widgets

def d1(S, K, T, r, sigma):
    """calculate d1 for Black-Scholes formula"""
    return (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))

def delta(S, K, T, r, sigma, option_type='call'):
    """calculate delta for European options"""
    d1_val = d1(S, K, T, r, sigma)
    if option_type == 'call':
        return si.norm.cdf(d1_val)
    else:
        return si.norm.cdf(d1_val) - 1

def gamma(S, K, T, r, sigma):
    """calculate gamma for European option."""
    d1_val = d1(S, K, T, r, sigma)
    return si.norm.pdf(d1_val) / (S * sigma * np.sqrt(T))

def theta(S, K, T, r, sigma, option_type='call'):
    """calculate theta for European options"""
    d1_val = d1(S, K, T, r, sigma)
    d2_val = d1_val - sigma * np.sqrt(T)
    term1 = -(S * si.norm.pdf(d1_val) * sigma) / (2 * np.sqrt(T))
    if option_type == 'call':
        term2 = r * K * np.exp(-r * T) * si.norm.cdf(d2_val)
        return term1 - term2
    else:
        term2 = r * K * np.exp(-r * T) * si.norm.cdf(-d2_val)
        return term1 + term2

def vega(S, K, T, r, sigma):
    """calculate vega for European options"""
    d1_val = d1(S, K, T, r, sigma)
    return S * si.norm.pdf(d1_val) * np.sqrt(T)

def rho(S, K, T, r, sigma, option_type='call'):
    """calculate rho for European options"""
    d2_val = d1(S, K, T, r, sigma) - sigma * np.sqrt(T)
    if option_type == 'call':
        return K * T * np.exp(-r * T) * si.norm.cdf(d2_val)
    else:
        return -K * T * np.exp(-r * T) * si.norm.cdf(-d2_val)

def vanna(S, K, T, r, sigma):
    """calculate vanna for European options"""
    d1_val = d1(S, K, T, r, sigma)
    vega_val = vega(S, K, T, r, sigma)
    return vega_val * (1 - d1_val / (sigma * np.sqrt(T)))

def charm(S, K, T, r, sigma, option_type='call'):
    """calculate charm for European options"""
    d1_val = d1(S, K, T, r, sigma)
    d2_val = d1_val - sigma * np.sqrt(T)
    factor = -si.norm.pdf(d1_val) * (2 * r * T - d2_val * sigma * np.sqrt(T)) / (2 * T * sigma * np.sqrt(T))
    return factor if option_type == 'call' else -factor

def plot_all_greeks(S_current, K, T, r, sigma, option_type, fixed_axes):
    """plot all greeks as a function of stock price with toggle for fixed axes"""
    S_range = np.linspace(50, 150, 400)
    greeks = {
        'Delta': [delta(S, K, T, r, sigma, option_type) for S in S_range],
        'Gamma': [gamma(S, K, T, r, sigma) for S in S_range],
        'Theta': [theta(S, K, T, r, sigma, option_type) for S in S_range],
        'Vega': [vega(S, K, T, r, sigma) for S in S_range],
        'Rho': [rho(S, K, T, r, sigma, option_type) for S in S_range],
        'Vanna': [vanna(S, K, T, r, sigma) for S in S_range],
        'Charm': [charm(S, K, T, r, sigma, option_type) for S in S_range]
    }

    current_values = {
        'Delta': delta(S_current, K, T, r, sigma, option_type),
        'Gamma': gamma(S_current, K, T, r, sigma),
        'Theta': theta(S_current, K, T, r, sigma, option_type),
        'Vega': vega(S_current, K, T, r, sigma),
        'Rho': rho(S_current, K, T, r, sigma, option_type),
        'Vanna': vanna(S_current, K, T, r, sigma),
        'Charm': charm(S_current, K, T, r, sigma, option_type)
    }

    # Reasonable fixed ranges based on typical behavior
    fixed_ranges = {
        'Delta': (-0.1, 1.1),
        'Gamma': (0, 0.1),
        'Theta': (-20, 0.1),
        'Vega': (0, 100),
        'Rho': (-1, 100),
        'Vanna': (-150, 150),
        'Charm': (-1, 1)
    }

    fig, axs = plt.subplots(4, 2, figsize=(14, 16))
    axs = axs.flatten()
    greek_names = list(greeks.keys())

    for i, greek in enumerate(greek_names):
        axs[i].plot(S_range, greeks[greek], label=f'{greek}', color='blue')
        axs[i].scatter(S_current, current_values[greek], color='red', zorder=5, label=f'Current {greek}: {current_values[greek]:.2f}')
        axs[i].axvline(S_current, color='red', linestyle='--', linewidth=0.5)
        axs[i].set_title(f'{greek} vs Stock Price')
        axs[i].set_ylabel(greek)
        axs[i].legend()
        axs[i].grid(True)

        # Only show the x-label "Stock Price (S)" on the bottom row
        if i >= 6:
            axs[i].set_xlabel('Stock Price (S)')

        # Set fixed or dynamic y-axis limits
        if fixed_axes:
            axs[i].set_ylim(fixed_ranges[greek])

    # Remove the last empty subplot
    fig.delaxes(axs[7])

    fig.tight_layout(pad=3.0)
    plt.show()

style = {'description_width': 'initial'}
widgets.interact(
    plot_all_greeks,
    S_current=widgets.FloatSlider(value=100, min=50, max=150, step=1, description='Stock Price (S)', style=style),
    K=widgets.FloatSlider(value=100, min=50, max=150, step=1, description='Strike Price (K)', style=style),
    T=widgets.FloatSlider(value=1, min=0.01, max=2, step=0.01, description='Time to Expiry (T)', style=style),
    r=widgets.FloatSlider(value=0.05, min=0, max=0.2, step=0.01, description='Risk-Free Rate (r)', style=style),
    sigma=widgets.FloatSlider(value=0.2, min=0.01, max=1, step=0.01, description='Volatility (σ)', style=style),
    option_type=widgets.Dropdown(options=['call', 'put'], value='call', description='Option Type', style=style),
    fixed_axes=widgets.Checkbox(value=True, description='Fixed Axes', style=style)
)


interactive(children=(FloatSlider(value=100.0, description='Stock Price (S)', max=150.0, min=50.0, step=1.0, s…

<function __main__.plot_all_greeks(S_current, K, T, r, sigma, option_type, fixed_axes)>